In [2]:
import pandas as pd
credits = pd.read_csv('credits.csv')
titles = pd.read_csv('titles.csv')

In [3]:
titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm89464,Miracle on 34th Street,MOVIE,"Kris Kringle, seemingly the embodiment of Sant...",1947,G,96,"['family', 'comedy', 'drama']",['US'],NaN,tt0039628,7.9,50969.0,23.515,7.388
1,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,G,68,"['horror', 'fantasy', 'animation', 'family', '...",['US'],NaN,tt0041094,6.9,16502.0,16.194,6.500
2,tm61052,Cinderella,MOVIE,Cinderella has faith her dreams of a better li...,1950,G,74,"['fantasy', 'animation', 'family', 'romance']",['US'],NaN,tt0042332,7.3,164292.0,93.547,7.035
3,tm87946,Dumbo,MOVIE,Dumbo is a baby elephant born with over-sized ...,1941,G,64,"['animation', 'drama', 'family', 'fantasy']",['US'],NaN,tt0033563,7.2,135045.0,54.265,6.999
4,tm74391,Fantasia,MOVIE,Walt Disney's timeless masterpiece is an extra...,1941,G,119,"['animation', 'family', 'fantasy', 'music']",['US'],NaN,tt0032455,7.7,98764.0,43.609,7.382


In [4]:
credits.head()

,person_id,id,name,character,role
0,35549,tm89464,Maureen O'Hara,Doris Walker,ACTOR
1,57832,tm89464,John Payne,Fred Gailey,ACTOR
2,57833,tm89464,Edmund Gwenn,Kris Kringle,ACTOR
3,25096,tm89464,Natalie Wood,Susan Walker,ACTOR
4,27185,tm89464,Porter Hall,Granville Sawyer,ACTOR


In [5]:
set(credits.role)

{'ACTOR', 'DIRECTOR'}

In [6]:
director = credits[credits['role'] == 'DIRECTOR']

In [7]:
director = director[['id','name']]

In [8]:
director

,id,name
65,tm89464,George Seaton
79,tm61729,James Algar
80,tm61729,Clyde Geronimi
81,tm61729,Jack Kinney
105,tm61052,Wilfred Jackson
...,...,...
30674,tm1314337,Brendan Walter
30675,tm1314337,Jacob Bixenman
30680,tm1299578,William Pinkney III
30687,tm1075680,Kevin Bachar


In [9]:
merged = pd.merge(titles, director, on = 'id')

In [10]:
merged = merged.rename(columns={'name': 'director'})

In [11]:
merged

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,director
0,tm89464,Miracle on 34th Street,MOVIE,"Kris Kringle, seemingly the embodiment of Sant...",1947,G,96,"['family', 'comedy', 'drama']",['US'],NaN,tt0039628,7.9,50969.0,23.515,7.388,George Seaton
1,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,G,68,"['horror', 'fantasy', 'animation', 'family', '...",['US'],NaN,tt0041094,6.9,16502.0,16.194,6.500,James Algar
2,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,G,68,"['horror', 'fantasy', 'animation', 'family', '...",['US'],NaN,tt0041094,6.9,16502.0,16.194,6.500,Clyde Geronimi
3,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,G,68,"['horror', 'fantasy', 'animation', 'family', '...",['US'],NaN,tt0041094,6.9,16502.0,16.194,6.500,Jack Kinney
4,tm61052,Cinderella,MOVIE,Cinderella has faith her dreams of a better li...,1950,G,74,"['fantasy', 'animation', 'family', 'romance']",['US'],NaN,tt0042332,7.3,164292.0,93.547,7.035,Wilfred Jackson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,tm1314337,Miley Cyrus – Endless Summer Vacation (Backyar...,MOVIE,"Miley Cyrus takes the stage in this must-see, ...",2023,PG-13,42,"['music', 'documentation']",['US'],NaN,tt27001634,NaN,NaN,10.543,10.000,Brendan Walter
1563,tm1314337,Miley Cyrus – Endless Summer Vacation (Backyar...,MOVIE,"Miley Cyrus takes the stage in this must-see, ...",2023,PG-13,42,"['music', 'documentation']",['US'],NaN,tt27001634,NaN,NaN,10.543,10.000,Jacob Bixenman
1564,tm1299578,Black Travel Across America,MOVIE,"Travel Consultant, Martinique Lewis, embarks o...",2023,PG,44,['documentation'],['US'],NaN,NaN,NaN,NaN,5.600,NaN,William Pinkney III
1565,tm1075680,World's Most Dangerous Shark?,MOVIE,"The Great White has a scary reputation, but Ex...",2023,PG-13,44,['documentation'],[],NaN,NaN,NaN,NaN,1.623,NaN,Kevin Bachar


In [12]:
merged.dtypes

id                       object
title                    object
type                     object
description              object
release_year              int64
age_certification        object
runtime                   int64
genres                   object
production_countries     object
seasons                 float64
imdb_id                  object
imdb_score              float64
imdb_votes              float64
tmdb_popularity         float64
tmdb_score              float64
director                 object
dtype: object

In [13]:
merged.isna().sum()

id                         0
title                      0
type                       0
description                2
release_year               0
age_certification        388
runtime                    0
genres                     0
production_countries       0
seasons                 1475
imdb_id                  467
imdb_score               495
imdb_votes               504
tmdb_popularity            0
tmdb_score                46
director                   0
dtype: int64

In [14]:
#see if NA value are from movie

print("All Movies have a seasons type of NA:")
print(merged.groupby(['type', 'seasons']).size())

print("\nAll movies have NA Value - 1475 movies, 1475 NA value for seasons:")
merged.groupby(['type']).size()

All Movies have a seasons type of NA:
type  seasons
SHOW  1.0        66
      2.0        10
      3.0        12
      4.0         2
      6.0         2
dtype: int64

All movies have NA Value - 1475 movies, 1475 NA value for seasons:


type
MOVIE    1475
SHOW       92
dtype: int64

In [15]:
#replace NA with zero for Movies
merged['seasons'].fillna(0, inplace=True)

#check
merged.isna().sum()

id                        0
title                     0
type                      0
description               2
release_year              0
age_certification       388
runtime                   0
genres                    0
production_countries      0
seasons                   0
imdb_id                 467
imdb_score              495
imdb_votes              504
tmdb_popularity           0
tmdb_score               46
director                  0
dtype: int64

In [16]:
#see more detail on null age_certification
merged[merged['age_certification'].isna()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,director
27,tm165060,The Tortoise and the Hare,MOVIE,The Tortoise and the Hare is an animated short...,1935,NaN,9,"['animation', 'documentation']",['US'],0.0,NaN,NaN,NaN,9.286,6.686,Wilfred Jackson
29,tm67239,Lonesome Ghosts,MOVIE,"On a dark and stormy night, four bored ghosts ...",1937,NaN,9,['comedy'],['US'],0.0,NaN,NaN,NaN,5.929,6.733,Burt Gillett
30,tm62671,Treasure Island,MOVIE,Enchanted by the idea of locating treasure bur...,1950,NaN,96,"['family', 'action']","['GB', 'US']",0.0,tt0043067,6.9,8772.0,8.872,6.500,Byron Haskin
38,tm87968,Early to Bed,MOVIE,"Donald has to get up early, but everything see...",1941,NaN,8,['animation'],['US'],0.0,NaN,NaN,NaN,4.764,6.713,Jack King
45,tm77407,Saludos Amigos,MOVIE,Live-action segments show members of the Disne...,1943,NaN,41,"['animation', 'comedy', 'family', 'fantasy']",['US'],0.0,tt0036326,6.1,6506.0,17.765,5.876,William Roberts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550,tm1239803,"Extra Episode, or Not: Behind the Scenes of Al...",MOVIE,What's it like to shoot a series that takes pl...,2022,NaN,29,"['documentation', 'comedy']",['BR'],0.0,tt18300554,NaN,NaN,0.600,NaN,Juliana Vonlanten
1551,ts314957,Mila in the Multiverse,SHOW,Mila is 16 years old and living the adventure ...,2023,NaN,26,"['scifi', 'action', 'comedy', 'drama']",['BR'],1.0,tt15551300,4.6,139.0,440.460,7.000,Júlia Pacheco Jordão
1554,tm1298257,Schoolhouse Rock! 50th Anniversary Singalong,MOVIE,Dust off your dictionaries and grab that hairb...,2023,NaN,40,"['comedy', 'family', 'history', 'music']",['US'],0.0,tt26231144,4.5,70.0,3.767,2.000,Lauren Quinn
1555,tm1298257,Schoolhouse Rock! 50th Anniversary Singalong,MOVIE,Dust off your dictionaries and grab that hairb...,2023,NaN,40,"['comedy', 'family', 'history', 'music']",['US'],0.0,tt26231144,4.5,70.0,3.767,2.000,Nick Florez


In [17]:
#replace NA with unrated for age_certification
merged['age_certification'].fillna('unrated', inplace=True)

#check
merged.isna().sum()

id                        0
title                     0
type                      0
description               2
release_year              0
age_certification         0
runtime                   0
genres                    0
production_countries      0
seasons                   0
imdb_id                 467
imdb_score              495
imdb_votes              504
tmdb_popularity           0
tmdb_score               46
director                  0
dtype: int64

In [18]:
#drop imdb since almost 30% is missing - will use tmdb instead

merged.drop('imdb_id', axis=1, inplace=True)
merged.drop('imdb_score', axis=1, inplace=True)
merged.drop('imdb_votes', axis=1, inplace=True)
merged.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,tmdb_popularity,tmdb_score,director
0,tm89464,Miracle on 34th Street,MOVIE,"Kris Kringle, seemingly the embodiment of Sant...",1947,G,96,"['family', 'comedy', 'drama']",['US'],0.0,23.515,7.388,George Seaton
1,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,G,68,"['horror', 'fantasy', 'animation', 'family', '...",['US'],0.0,16.194,6.500,James Algar
2,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,G,68,"['horror', 'fantasy', 'animation', 'family', '...",['US'],0.0,16.194,6.500,Clyde Geronimi
3,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,G,68,"['horror', 'fantasy', 'animation', 'family', '...",['US'],0.0,16.194,6.500,Jack Kinney
4,tm61052,Cinderella,MOVIE,Cinderella has faith her dreams of a better li...,1950,G,74,"['fantasy', 'animation', 'family', 'romance']",['US'],0.0,93.547,7.035,Wilfred Jackson


In [ ]:
#I do age and genre

In [ ]:
#Genre

In [24]:
#remove square brackets, quotations and spaces from genres

merged['genres'] = pd.DataFrame([str(line).strip('[').strip(']') for line in merged['genres']])
merged['genres'] = merged['genres'].str.replace(r"'", r"")
merged['genres'] = merged['genres'].str.replace(r" ", r"")
merged.head(2)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,tmdb_popularity,tmdb_score,director
0,tm89464,Miracle on 34th Street,MOVIE,"Kris Kringle, seemingly the embodiment of Sant...",1947,G,96,"family,comedy,drama",['US'],0.0,23.515,7.388,George Seaton
1,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,G,68,"horror,fantasy,animation,family,comedy",['US'],0.0,16.194,6.500,James Algar


In [25]:
#get dummies for genres

dummies = merged['genres'].str.get_dummies(sep=',')
dummies.head(5)

,action,animation,comedy,crime,documentation,drama,european,family,fantasy,history,horror,music,reality,romance,scifi,sport,thriller,war,western
0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0


In [26]:
#concatenate dummies with genres back into merged
merged2 = (pd.concat([merged, dummies], axis=1).drop(["genres"], axis=1))
print(merged2.columns)

Index(['id', 'title', 'type', 'description', 'release_year',
       'age_certification', 'runtime', 'production_countries', 'seasons',
       'tmdb_popularity', 'tmdb_score', 'director', 'action', 'animation',
       'comedy', 'crime', 'documentation', 'drama', 'european', 'family',
       'fantasy', 'history', 'horror', 'music', 'reality', 'romance', 'scifi',
       'sport', 'thriller', 'war', 'western'],
      dtype='object')


In [27]:
#get dummies for age_certification

dummies2 = merged['age_certification'].str.get_dummies(sep=',')
dummies2.head(5)

,G,PG,PG-13,R,TV-14,TV-G,TV-PG,TV-Y,TV-Y7,unrated
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0


In [28]:
#get count by age_certification

dummies2_sum = merged['age_certification'].str.get_dummies(sep=',').sum()
dummies2_sum

G          545
PG         431
PG-13      136
R            5
TV-14       10
TV-G        17
TV-PG       21
TV-Y         8
TV-Y7        6
unrated    388
dtype: int64

In [29]:
#concatenate dummies2 with age_certification back into merged2
merged3 = (pd.concat([merged2, dummies2], axis=1).drop(["age_certification"], axis=1))
print(merged3.columns)

Index(['id', 'title', 'type', 'description', 'release_year', 'runtime',
       'production_countries', 'seasons', 'tmdb_popularity', 'tmdb_score',
       'director', 'action', 'animation', 'comedy', 'crime', 'documentation',
       'drama', 'european', 'family', 'fantasy', 'history', 'horror', 'music',
       'reality', 'romance', 'scifi', 'sport', 'thriller', 'war', 'western',
       'G', 'PG', 'PG-13', 'R', 'TV-14', 'TV-G', 'TV-PG', 'TV-Y', 'TV-Y7',
       'unrated'],
      dtype='object')


In [31]:
#remove quotation marks and brackets from country from genres
merged3['production_countries'] = pd.DataFrame([str(line).strip('[').strip(']') for line in merged3['production_countries']])
merged3['production_countries'] = merged3['production_countries'].str.replace(r"'", r"")
merged3['production_countries'] = merged3['production_countries'].str.replace(r" ", r"")
merged3.head(2)

,id,title,type,description,release_year,runtime,production_countries,seasons,tmdb_popularity,tmdb_score,...,G,PG,PG-13,R,TV-14,TV-G,TV-PG,TV-Y,TV-Y7,unrated
0,tm89464,Miracle on 34th Street,MOVIE,"Kris Kringle, seemingly the embodiment of Sant...",1947,96,US,0.0,23.515,7.388,...,1,0,0,0,0,0,0,0,0,0
1,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,68,US,0.0,16.194,6.500,...,1,0,0,0,0,0,0,0,0,0


In [32]:
#get dummies for production_countries

dummies3 = merged3['production_countries'].str.get_dummies(sep=',')
dummies3.columns

Index(['AE', 'AR', 'AT', 'AU', 'BE', 'BR', 'BW', 'CA', 'CH', 'CI', 'CL', 'CN',
       'CO', 'CZ', 'DE', 'DK', 'ES', 'FR', 'GB', 'GR', 'HK', 'IE', 'IN', 'IT',
       'JP', 'KI', 'KR', 'MC', 'MX', 'NL', 'NO', 'NZ', 'PH', 'PL', 'SE', 'SI',
       'SK', 'TR', 'TW', 'UG', 'US', 'ZA'],
      dtype='object')

In [33]:
#get count by production_countries

dummies3_sum = merged3['production_countries'].str.get_dummies(sep=',').sum()
dummies3_sum

AE       1
AR       9
AT       2
AU      24
BE       2
BR       9
BW       2
CA      57
CH       1
CI       2
CL       1
CN       7
CO       1
CZ       1
DE      16
DK       4
ES       4
FR      28
GB      71
GR       1
HK       2
IE       1
IN       8
IT       7
JP       5
KI       1
KR       5
MC       1
MX       2
NL       1
NO       1
NZ       5
PH       1
PL       1
SE       1
SI       1
SK       1
TR       2
TW       1
UG       2
US    1431
ZA       1
dtype: int64

In [34]:
dummies3.head()

,AE,AR,AT,AU,BE,BR,BW,CA,CH,CI,...,PH,PL,SE,SI,SK,TR,TW,UG,US,ZA
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
#Popularity based on Genre Regression Model

In [35]:
genre_mlr = merged3
genre_mlr.head()

,id,title,type,description,release_year,runtime,production_countries,seasons,tmdb_popularity,tmdb_score,...,G,PG,PG-13,R,TV-14,TV-G,TV-PG,TV-Y,TV-Y7,unrated
0,tm89464,Miracle on 34th Street,MOVIE,"Kris Kringle, seemingly the embodiment of Sant...",1947,96,US,0.0,23.515,7.388,...,1,0,0,0,0,0,0,0,0,0
1,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,68,US,0.0,16.194,6.500,...,1,0,0,0,0,0,0,0,0,0
2,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,68,US,0.0,16.194,6.500,...,1,0,0,0,0,0,0,0,0,0
3,tm61729,The Adventures of Ichabod and Mr. Toad,MOVIE,The Wind in the Willows: Concise version of Ke...,1950,68,US,0.0,16.194,6.500,...,1,0,0,0,0,0,0,0,0,0
4,tm61052,Cinderella,MOVIE,Cinderella has faith her dreams of a better li...,1950,74,US,0.0,93.547,7.035,...,1,0,0,0,0,0,0,0,0,0


In [36]:
genre_mlr.columns

Index(['id', 'title', 'type', 'description', 'release_year', 'runtime',
       'production_countries', 'seasons', 'tmdb_popularity', 'tmdb_score',
       'director', 'action', 'animation', 'comedy', 'crime', 'documentation',
       'drama', 'european', 'family', 'fantasy', 'history', 'horror', 'music',
       'reality', 'romance', 'scifi', 'sport', 'thriller', 'war', 'western',
       'G', 'PG', 'PG-13', 'R', 'TV-14', 'TV-G', 'TV-PG', 'TV-Y', 'TV-Y7',
       'unrated'],
      dtype='object')

In [37]:
#Divide the data set training and testing with a (70, 30) rule. 

from sklearn.model_selection import train_test_split

x = genre_mlr[['action', 'animation', 'comedy', 'crime', 'documentation',
       'drama', 'european', 'family', 'fantasy', 'history', 'horror', 'music',
       'reality', 'romance', 'scifi', 'sport', 'thriller', 'war', 'western']]

y = genre_mlr.tmdb_popularity

x_train, x_test, y_train, y_test = train_test_split(x, y ,test_size=0.30, random_state=51)

In [39]:
#Linear Regression

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [40]:
RLR = LinearRegression().fit(x_train, y_train)
RLR_ytrain_pred = RLR.predict(x_train)
RLR_ytest_pred = RLR.predict(x_test)

In [41]:
#view head of training dataset
print('RLR Training Dataset: \n')
pd.DataFrame(list(zip(y_train, RLR_ytrain_pred)), columns=['ytrain','ytrainhat']).head(5)

RLR Training Dataset: 



,ytrain,ytrainhat
0,4.088,42.827401
1,6.464,7.268199
2,106.037,65.489421
3,7.868,22.424573
4,4.520,22.424573


In [43]:
#R-squared - % of variability explained in model

import sklearn.metrics as metrics


RLR_testr2 = metrics.r2_score(y_test, RLR_ytest_pred).round(6)
RLR_trainr2 = metrics.r2_score(y_train, RLR_ytrain_pred).round(6)
print("RLR train vs test R^2:",RLR_trainr2, RLR_testr2)

RLR train vs test R^2: 0.11379 0.056376


In [ ]:
#divide dataset based on year and create model

In [ ]:
#should we look at popularity by release year - popularity and rating don't always match